In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 6
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 27), datetime.date(2023, 6, 26))

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-06-27
yesterday: 2023-06-26


### Restart and Run All Cells

## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(35, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(192, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(35, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(35, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,BANPU,-22,RD30pct,SET50,3000,7.8,8.9,-0.1,-1.111111,0,2023-04-10
B,BBL,-73,,SET50,200,125.0,161.5,-1.5,-0.920245,0,2023-04-21
B,BDMS,-22,XXX,SET50,1000,24.0,28.0,-0.5,-1.754386,0,2022-12-22
B,BEM,-8,52WL,SET50,2500,7.8,8.2,-0.05,-0.606061,1,2022-12-22
B,BH,-87,5pct,SET50,200,169.0,225.0,-1.0,-0.442478,0,2022-12-22
B,CPNREIT,-12,5pct,SET,5000,10.2,11.4,0.7,6.542056,0,2023-05-23
B,GVREIT,-5,RD10pct,SET,10000,8.1,8.35,0.05,0.60241,1,2023-05-26
B,JMART,-12,RD55pct,SET50,1800,12.5,13.7,0.6,4.580153,0,2023-04-11
B,JMT,-7,RD35pct,SET50,900,33.0,34.75,1.25,3.731343,1,2023-04-11
B,KKP,-7,,SET100,500,58.0,59.75,0.0,0.0,1,2023-04-28
B,KTB,-50,,SET50,1500,14.3,19.3,-0.2,-1.025641,0,2023-04-20
B,LANNA,-12,,SET,1500,13.8,15.0,0.0,0.0,0,2023-03-18
B,MCS,-9,RD60pct,SET,4000,6.0,6.45,-0.2,-3.007519,1,2023-03-17
B,NER,-6,RD40pct,SET,5000,4.4,4.52,-0.02,-0.440529,1,2023-04-20
B,ORI,-10,RD15pct,SET100,2000,9.5,10.0,-0.1,-0.99009

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
15,B,PTG,-2,NaN,SET100,1800,11.40,11.60,0.00,0.00,1,2022-12-22
20,B,TFFIF,-2,DOS,SET,10000,7.50,7.60,0.05,0.66,1,2023-05-31


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
26,S,DIF,5,5pct,SET,2000,11.20,10.70,0.10,0.94,2,2023-05-15
27,S,EA,12,5pct,SET50,400,55.75,52.75,0.25,0.48,2,2022-12-22
34,S,WHART,6,5pct,SET,10000,11.00,10.40,-0.20,-1.89,2,2022-02-02


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
3,B,BEM,-8,52WL,SET50,2500,7.80,8.20,-0.05,-0.61,1,2022-12-22
6,B,GVREIT,-5,RD10pct,SET,10000,8.10,8.35,0.05,0.60,1,2023-05-26
8,B,JMT,-7,RD35pct,SET50,900,33.00,34.75,1.25,3.73,1,2023-04-11
9,B,KKP,-7,NaN,SET100,500,58.00,59.75,0.00,0.00,1,2023-04-28
12,B,MCS,-9,RD60pct,SET,4000,6.00,6.45,-0.20,-3.01,1,2023-03-17
...,...,...,...,...,...,...,...,...,...,...,...,...
17,B,STA,-6,RD60pct,SET100,1500,16.20,16.80,-0.20,-1.18,1,2023-04-19
18,B,SYNEX,-6,RND60pct,SET,2000,11.70,12.30,0.20,1.65,1,2023-03-09
20,B,TFFIF,-2,DOS,SET,10000,7.50,7.60,0.05,0.66,1,2023-05-31
21,B,TMT,-7,RD30pct,SET,3000,6.95,7.30,0.05,0.69,1,2023-04-18


In [17]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
0,B,BANPU,-22,RD30pct,SET50,3000,7.80,8.90,-0.10,-1.11,0,2023-04-10
1,B,BBL,-73,NaN,SET50,200,125.00,161.50,-1.50,-0.92,0,2023-04-21
2,B,BDMS,-22,XXX,SET50,1000,24.00,28.00,-0.50,-1.75,0,2022-12-22
4,B,BH,-87,5pct,SET50,200,169.00,225.00,-1.00,-0.44,0,2022-12-22
5,B,CPNREIT,-12,5pct,SET,5000,10.20,11.40,0.70,6.54,0,2023-05-23
...,...,...,...,...,...,...,...,...,...,...,...,...
28,S,ICHI,21,5pct,SET,1500,15.00,12.90,-0.30,-2.27,0,2022-12-22
29,S,IVL,21,5pct,SET50,2400,37.75,32.50,-1.00,-2.99,0,2023-05-26
31,S,KTC,11,5pct,SET50,900,52.50,49.75,0.00,0.00,0,2022-12-22
32,S,PTTGC,68,RD45pct,SET50,600,53.25,36.25,-0.50,-1.36,0,2023-02-27


In [18]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate


In [19]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [20]:
df['percent'] = df['%change']

In [21]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,percent
12,B,MCS,-9,RD60pct,SET,4000,6.00,6.45,-0.20,-3.01,1,2023-03-17,-3.01
28,S,ICHI,21,5pct,SET,1500,15.00,12.90,-0.30,-2.27,0,2022-12-22,-2.27
29,S,IVL,21,5pct,SET50,2400,37.75,32.50,-1.00,-2.99,0,2023-05-26,-2.99


In [22]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,percent
5,B,CPNREIT,-12,5pct,SET,5000,10.20,11.40,0.70,6.54,0,2023-05-23,6.54
7,B,JMART,-12,RD55pct,SET50,1800,12.50,13.70,0.60,4.58,0,2023-04-11,4.58
8,B,JMT,-7,RD35pct,SET50,900,33.00,34.75,1.25,3.73,1,2023-04-11,3.73
23,S,AH,27,5pct,SET,1200,39.00,32.25,1.75,5.74,0,2022-12-22,5.74


In [23]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,percent
20,B,TFFIF,-2,DOS,SET,10000,7.50,7.60,0.05,0.66,1,2023-05-31,0.66
15,B,PTG,-2,NaN,SET100,1800,11.40,11.60,0.00,0.00,1,2022-12-22,0.00
6,B,GVREIT,-5,RD10pct,SET,10000,8.10,8.35,0.05,0.60,1,2023-05-26,0.60
18,B,SYNEX,-6,RND60pct,SET,2000,11.70,12.30,0.20,1.65,1,2023-03-09,1.65
17,B,STA,-6,RD60pct,SET100,1500,16.20,16.80,-0.20,-1.18,1,2023-04-19,-1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,B,TASCO,-25,XXX,SET,1500,15.60,18.10,0.00,0.00,0,2022-12-22,0.00
10,B,KTB,-50,NaN,SET50,1500,14.30,19.30,-0.20,-1.03,0,2023-04-20,-1.03
22,B,TTB,-50,52WL,SET50,20000,1.09,1.59,-0.01,-0.62,0,2023-04-18,-0.62
1,B,BBL,-73,NaN,SET50,200,125.00,161.50,-1.50,-0.92,0,2023-04-21,-0.92


### Sell recently purchased stocks if profitable

In [24]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(112, 4)

In [25]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [26]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(111, 8)

In [27]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
21,DIF,2023-06-23,2000,10.60,10.70,200.00,0.94,0.94


### Profit per Day

In [28]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 6
ORDER BY sells.date DESC, name


In [29]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,CPNREIT,2023,6,10.70,10.20,0.50,"3,000","32,100.00","30,600.00","1,500.00",4.90,3
1,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,17
2,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,28
3,NER,2023,6,4.84,4.60,0.24,"5,000","24,200.00","23,000.00","1,200.00",5.22,15
4,JMT,2023,6,41.75,39.50,2.25,600,"25,050.00","23,700.00","1,350.00",5.70,3
5,ORI,2023,6,11.20,10.20,1.00,"3,000","33,600.00","30,600.00","3,000.00",9.80,23
6,EA,2023,6,64.00,61.00,3.00,400,"25,600.00","24,400.00","1,200.00",4.92,7
7,KTC,2023,6,52.50,50.00,2.50,400,"21,000.00","20,000.00","1,000.00",5.00,1


In [30]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [31]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,CPNREIT,2023,6,10.70,10.20,0.50,"3,000","32,100.00","30,600.00","1,500.00",4.90,3,1.633987
1,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,17,0.038447
2,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,28,0.023343
3,NER,2023,6,4.84,4.60,0.24,"5,000","24,200.00","23,000.00","1,200.00",5.22,15,0.347826
4,JMT,2023,6,41.75,39.50,2.25,600,"25,050.00","23,700.00","1,350.00",5.70,3,1.898734
5,ORI,2023,6,11.20,10.20,1.00,"3,000","33,600.00","30,600.00","3,000.00",9.80,23,0.426257
6,EA,2023,6,64.00,61.00,3.00,400,"25,600.00","24,400.00","1,200.00",4.92,7,0.702576
7,KTC,2023,6,52.50,50.00,2.50,400,"21,000.00","20,000.00","1,000.00",5.00,1,5.000000


In [32]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 1.26%
